# Visualize DVS Input

This notebook demonstrates how to visualize the DVS sensor's input on the speck2e devkit.

In [1]:
import samna, samnagui
import time
import sys
import os

from multiprocessing import Process

## 1. Open devkit

In [2]:
# list all the device we plugged to our PC
devices = samna.device.get_unopened_devices()
device_names = [each.device_type_name for each in devices]
print("the devices are: ", device_names)

the devices are:  ['Speck2eDevKit']


In [3]:
# when open devkit, we just need to pass the device name to the `open_device` function of samna.device
devkit = samna.device.open_device("Speck2eDevKit")

## 2. Build samna graph

samna graph defines how the data flow into and out from the devkit. In this case, we wanted to send the raw DVS input events to a [`VizEventStreamer`](https://synsense-sys-int.gitlab.io/samna/filters.html?highlight=streamer#VizEventStreamer) node which can help us to visualize them.

![samna_graph](../../_static/visualize_speck_dvs/samna_graph.png)

### 2.1 Get the source node of the devkit

The model source node ties into the filter system. It will emit all events produced by the device.

In [4]:
# get the device source node of devkit
dk_source_node = devkit.get_model_source_node()
dk_source_node

### 2.2 Init samna graph and VizEventStreamer node

In [5]:
def build_samna_graph(dk_source_node):
    
    # build a graph in samna to show dvs
    samna_graph = samna.graph.EventFilterGraph()
    _, _, streamer_node = samna_graph.sequential([dk_source_node, "Speck2eDvsToVizConverter", "VizEventStreamer"])
    
    return samna_graph, streamer_node

In [6]:
# init samna graph and VisEventStreamer node
samna_graph, streamer_node = build_samna_graph(dk_source_node)

# besides, we need to set the tcp port of the streamer node
streamer_endpoint = "tcp://0.0.0.0:40000"
streamer_node.set_streamer_endpoint(streamer_endpoint)

## 3. Create a sub-process for GUI visualization


### 3.1 init samna node and get endpoints related with the visualizer

In [7]:
# init samna node
samna_node = samna.init_samna()
time.sleep(1)

# get endpoints
sender_endpoint = samna_node.get_sender_endpoint()
receiver_endpoint = samna_node.get_receiver_endpoint()

print(f"sender endpoint is: {sender_endpoint}")
print(f"recevier endpoint is: {receiver_endpoint}")

sender_endpoint: tcp://0.0.0.0:43321
receiver_endpoint: tcp://0.0.0.0:50633
sender endpoint is: tcp://0.0.0.0:43321
recevier endpoint is: tcp://0.0.0.0:50633


### 3.2 create a sub-process which runs the GUI visualizer

`samnagui` provide us a GUI visualizer for us to visualize the DVS inputs. We can start a visualizer by calling [`samnagui.runVisualizer`](https://synsense-sys-int.gitlab.io/samna/gui/index.html?highlight=runvisualizer#samna.runVisualizer)

In [8]:
def run_samnagui_visualizer(receiver_endpoint, sender_endpoint, visualizer_id):
    
    # define the width and height for the GUI window
    # width and height is represented by the ratio between window width and the width of main monitor
    width, height = 0.6, 0.6
    
    # start visualizer
    samnagui.runVisualizer(width, height, receiver_endpoint, sender_endpoint, visualizer_id)
    
    return
    

then we create a sub-process which runs the `run_samnagui_visualizer` function.

In [9]:
visualizer_id = 3
gui_process = Process(target=run_samnagui_visualizer, args=(receiver_endpoint, sender_endpoint, visualizer_id))

start the GUI sub-process:

In [10]:
gui_process.start()

# wait for open visualizer and connect to it.
timeout = 10
begin = time.time()
name = "visualizer" + str(visualizer_id)

while time.time() - begin < timeout:
    try:
        time.sleep(0.05)
        samna.open_remote_node(visualizer_id, name)
    except:
        continue
    else:
        print("successfully open the visualizer")
        break

successfully open the visualizer


### 3.3 setup the visualizer 

In [11]:
# get visualizer object
visualizer = getattr(samna, name)

#  
activity_plot_id = visualizer.plots.add_activity_plot(128, 128, "DVS Layer")
plot_name = "plot_" + str(activity_plot_id)
plot = getattr(visualizer, plot_name)
# set the position: top left x, top left y, bottom right x, bottom right y
plot.set_layout(0, 0, 0.6, 1)   


# setup the visualizer
# set the recevier endpoint of visualizer as the streamer node's endpoint
visualizer.receiver.set_receiver_endpoint(streamer_endpoint)
visualizer.receiver.add_destination(visualizer.splitter.get_input_channel())
visualizer.splitter.add_destination("passthrough", visualizer.plots.get_plot_input(activity_plot_id))

True

## 4. Launch the devkit and start visualization

### 4.1 start samna graph

In [12]:
samna_graph.start()

True

### 4.2 create devkit config and apply to devkit

the [configuration](https://synsense-sys-int.gitlab.io/samna/reference/speck2e/configuration/index.html) of Speck2e devkit has an attribute named `dvs_layer` which is an instance of [`samna.speck2e.configuration.DvsLayerConfig`](https://synsense-sys-int.gitlab.io/samna/reference/speck2e/configuration/index.html?highlight=dvslayerconfig#samna.speck2e.configuration.DvsLayerConfig).

there are two attribute for a `DvsLayerConfig` we can set to be `True` if we wanted to monitor the input spikes from the dvs sensor:

1. [`monitor_enable`](https://synsense-sys-int.gitlab.io/samna/reference/speck2e/configuration/index.html?highlight=monitor_enable#samna.speck2e.configuration.DvsLayerConfig.monitor_enable): which let us monitor the pre-processing block of the devkit.

2. [`raw_monitor_enable`](https://synsense-sys-int.gitlab.io/samna/reference/speck2e/configuration/index.html?highlight=raw_monitor_enable#samna.speck2e.configuration.DvsLayerConfig.raw_monitor_enable): which let us monitor the raw input from the dvs sensor.

In [13]:
# modify configuration
devkit_config = samna.speck2e.configuration.SpeckConfiguration()
# enable monitoring the inputs from the DVS sensor 
devkit_config.dvs_layer.raw_monitor_enable = True
devkit.get_model().apply_configuration(devkit_config)

## 5. Stop the devkit and visualization

In [14]:
gui_process.terminate()
gui_process.join()

samna_graph.stop()

True